In [30]:
import torch
from transformers import pipeline

In [23]:

classifier = pipeline("sentiment-analysis")
classifier("We are very happy to show you the 🤗 Transformers library.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998}]

In [28]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])

In [29]:
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


In [33]:
from datasets import load_dataset, Audio
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset minds14 downloaded and prepared to /home/xyin/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/ca170f4e2acf536108fbb62103a210428d54744476a461115e41d4b2b57b185f. Subsequent calls will reuse this data.


In [35]:
speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FODING HOW I'D SET UP A JOIN TO HET WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE AP SO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AND I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I THURN A JOIN A COUNT']


In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [42]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

[{'label': '5 stars', 'score': 0.7272651791572571}]

In [41]:
classifier("我真的栓Q")

[{'label': '5 stars', 'score': 0.44867512583732605}]

In [1]:
from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

print(summary(model))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           1,538
Total params: 10

In [49]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

In [50]:
from transformers import DistilBertConfig

config = DistilBertConfig()

In [51]:
my_config = DistilBertConfig(activation="relu", attention_dropout=0.4)
print(my_config)

DistilBertConfig {
  "activation": "relu",
  "attention_dropout": 0.4,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}



In [52]:
from transformers import DistilBertModel

model = DistilBertModel(my_config)
print(model.num_parameters())

66362880


In [56]:
summary(model)

Layer (type:depth-idx)                             Param #
DistilBertModel                                    --
├─Embeddings: 1-1                                  --
│    └─Embedding: 2-1                              23,440,896
│    └─Embedding: 2-2                              393,216
│    └─LayerNorm: 2-3                              1,536
│    └─Dropout: 2-4                                --
├─Transformer: 1-2                                 --
│    └─ModuleList: 2-5                             --
│    │    └─TransformerBlock: 3-1                  7,087,872
│    │    └─TransformerBlock: 3-2                  7,087,872
│    │    └─TransformerBlock: 3-3                  7,087,872
│    │    └─TransformerBlock: 3-4                  7,087,872
│    │    └─TransformerBlock: 3-5                  7,087,872
│    │    └─TransformerBlock: 3-6                  7,087,872
Total params: 66,362,880
Trainable params: 66,362,880
Non-trainable params: 0

In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("wmt16", "ro-en")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /home/xyin/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/28ebdf8cf22106c2f1e58b2083d4b103608acd7bfdb6b14313ccd9e5bc8c313a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_datasets["train"][0]
# 我们可以看到一句英语en对应一句罗马尼亚语言ro
# {'translation': {'en': 'Membership of Parliament: see Minutes','ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [4]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'In the past, because of the cold shower received by Turkey at the Luxembourg summit in 1997, and today because of the stalled accession negotiations.""', 'ro': 'În trecut, din cauza duşului rece primit de Turcia la summitul de la Luxemburg din 1997 şi în prezent, deoarece negocierile de aderare stagnează"".'}"
1,"{'en': 'Therefore, it is necessary to develop a new approach to economic governance, to create a true economic union, to improve coordination and to harmonise financial and even fiscal policy.', 'ro': 'Prin urmare, este necesară dezvoltarea unei abordări noi față de guvernanța economică, pentru a crea o uniune economică reală, pentru a îmbunătăți coordonarea și a armoniza politica financiară și chiar cea fiscală.'}"
2,"{'en': 'The Lado ensemble, specialising in Croatian folk dances and songs, began a five-week tour in Japan.', 'ro': 'Ansamblul Lado, specializat în dansuri şi cântece populare croate, a demarat un turneu de cinci săptămâni în Japonia.'}"
3,"{'en': 'by Georgios Stavrakakis, on behalf of the Committee on Budgetary Control, on discharge in respect of the implementation of the budget of the SESAR Joint Undertaking for the financial year 2009 (SEC(2010)0963 - C7-0248/2010 -.', 'ro': 'elaborat de Georgios Stavrakakis, în numele Comisiei pentru control bugetar, referitor la descărcarea de gestiune pentru execuția bugetului Întreprinderii comune SESAR aferent exercițiului financiar 2009 [SEC(2010)0963 - C7-0248/2010 -.'}"
4,"{'en': 'The RS government proposed this law because the relevant commissions dealing with the division of state property in BiH have been unable to agree since, said Justice Minister Gerard Selman.', 'ro': 'Guvernul RS a propus această lege deoarece comisiile relevante care se ocupă de împărţirea proprietăţii de stat în BiH nu au reuşit să cadă de acord de atunci, a declarat ministrul justiţiei Gerard Selman.'}"


In [7]:
from transformers import AutoTokenizer
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro" 
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.